In [166]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    AddrExtractor,

    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [167]:
def get_ner(incident):
    doc = Doc(incident)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.parse_syntax(syntax_parser)

    doc.tag_ner(ner_tagger)

    for span in doc.spans:
        span.normalize(morph_vocab)
    
    for span in doc.spans:
        if span.type == PER:
            span.extract_fact(names_extractor)

    persons = []

    for person in doc.spans:
        if person.type == PER:
            person_dict = person.fact.as_dict
            if list(person_dict.keys()) == ['first']:
                persons.append(person_dict['first'])
            if list(person_dict.keys()) == ['first', 'last']:
                persons.append(person_dict['first'] + ' ' + person_dict['last'])
            if list(person_dict.keys()) == ['first', 'last', 'middle']:
                persons.append(person_dict['first'] + ' ' + person_dict['middle'] + ' ' +person_dict['last'])
    
    addrs = []
    
    if addr_extractor.find(incident) != None:
        for addr in addr_extractor.find(incident).fact.parts:
            addrs.append(addr.type + ' ' + addr.value)
    
    print([persons, addrs])